Code snippets that load the meta-model.
(And then you can test etc)

In [1]:
import os
#os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.6"  # preallocate a bit less memory so we can use pytorch next to jax

from time import time
import jax
import jax.numpy as jnp
import numpy as np
import optax
from typing import List, Dict
from jax.typing import ArrayLike
from meta_transformer import utils, preprocessing, torch_utils, module_path, on_cluster, output_dir
from meta_transformer.meta_model import MetaModel
import wandb
import argparse
from dataclasses import asdict
from meta_transformer.train import Updater, Logger
from meta_transformer.data import data_iterator, split_data
from etils import epath
import haiku as hk
from meta_transformer.transformer import Transformer
import flax

/home/lauro/.virtualenvs/meta-models/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rng = jax.random.PRNGKey(0)
x = jax.random.normal(rng, (1, 5, 10))

try:
    model = MetaModel(
        d_model=400,
        num_heads=2,
        num_layers=1,
        dropout_rate=0.1,
        widening_factor=4,
        use_embedding=True,
    )

except ValueError as err: # haiku
    @hk.transform
    def model(x, is_training):
        model_fn = MetaModel(
            model_size=400,
            use_embedding=True,
            transformer=Transformer(
                num_layers=1,
                num_heads=2,
                dropout_rate=0.1,
                key_size=400//2,
                widening_factor=4,
            )
        )
        return model_fn(x, is_training=is_training)

params = model.init(rng, x, is_training=False)




def filter_key(key: str):
    to_remove = ["bias", "LayerNorm", "layer_norm", "positional", "/b", "unembed"]
    return any(s in key for s in to_remove)


params = flax.traverse_util.flatten_dict(params, sep="/")
params = {k: np.array(v) for k, v in params.items() if not filter_key(k)}

In [3]:
# import json_tricks as json
# 
# def save_dict_to_json(dictionary, filename):
#     with open(filename, 'w') as file:
#         json.dump(dictionary, file, indent=4)
# 
# def load_dict_from_json(filename):
#     with open(filename, 'r') as file:
#         return json.load(file)
# mup_params_name = "mup_flax_params.json"
# sp_params_name = "sp_haiku_params.json"
# 
# 
# 
# #save_dict_to_json(params, mup_params_name)
# #save_dict_to_json(params, sp_params_name)
# mup_params = load_dict_from_json(mup_params_name)
# sp_params = load_dict_from_json(sp_params_name)
# sp_vars = jax.tree_map(lambda w: jnp.var(w).item(), sp_params)

In [4]:
# print(*sp_vars.keys(), sep='\n')
# print(*mup_vars.keys(), sep='\n')

In [5]:
sp_vars = {
 'params/input/embedding/kernel': 0.07694920897483826,
 'params/transformer/TransformerBlock_0/self_attention/query/kernel': 0.004992244299501181,
 'params/transformer/TransformerBlock_0/self_attention/key/kernel': 0.004987955093383789,
 'params/transformer/TransformerBlock_0/self_attention/value/kernel': 0.005008809268474579,
 'params/transformer/TransformerBlock_0/self_attention/out/kernel': 0.005000918637961149,
 'params/transformer/TransformerBlock_0/Dense_0/kernel': 0.005005593877285719,
 'params/transformer/TransformerBlock_0/Dense_1/kernel': 0.0012498314026743174
 }

In [6]:
mup_vars = jax.tree_map(lambda w: jnp.var(w).item(), params)
mup_vars = dict(mup_vars)

In [7]:
jax.tree_map(lambda x, y: x / y, sp_vars, mup_vars)

{'params/input/embedding/kernel': 1.007582270365108,
 'params/transformer/TransformerBlock_0/Dense_0/kernel': 1.0015089579236829,
 'params/transformer/TransformerBlock_0/Dense_1/kernel': 0.9982550014937603,
 'params/transformer/TransformerBlock_0/self_attention/key/kernel': 0.9978120249202611,
 'params/transformer/TransformerBlock_0/self_attention/out/kernel': 1.0042500702033048,
 'params/transformer/TransformerBlock_0/self_attention/query/kernel': 1.0010213052033996,
 'params/transformer/TransformerBlock_0/self_attention/value/kernel': 0.9983570689426814}

In [8]:
slkfdsk

NameError: name 'slkfdsk' is not defined

In [ ]:
stds = jax.tree_map(lambda w: jnp.std(w), loaded_params)
stds = {k: v.item() for k, v in stds.items()}
stds = {k: round(v, 5) for k, v in stds.items()}
stds

{'meta_model/linear/w': 0.2774,
 'transformer/multi_head_attention/query/w': 0.07066,
 'transformer/multi_head_attention/key/w': 0.07063,
 'transformer/multi_head_attention/value/w': 0.07077,
 'transformer/multi_head_attention/linear/w': 0.07072,
 'transformer/linear/w': 0.07075,
 'transformer/linear_1/w': 0.03535,
 'meta_model/linear_1/w': 0.04401}